In [24]:
import json
import geopandas as gpd
from django.contrib.gis.geos import GEOSGeometry
from shapely.geometry import mapping

f_path = './export_rnb.gpkg'

In [6]:
# Get all layers list
layers = gpd.io.file.fiona.listlayers(f_path)
print(layers)


['toit', 'aretes_toit']


In [39]:
gdf = gpd.read_file(f_path, layer="toit")

print('SRID')
srid = gdf.crs.to_epsg()
print(srid)

print("Get a sneakpeak of the data")
print(gdf.head())

print("Rows count")
print(len(gdf))



SRID
3946
Get a sneakpeak of the data
   gid  fid     id        parcelle       typetoit  hfacade  ztoitmini  \
0  NaN    1  42119  69259000CA0036  Toit non plan      5.6      217.0   
1  NaN    1  37869  69259000CE0104      Toit plan     20.2      231.8   
2  NaN    1  37885  69259000BY0082  Toit non plan      4.4      207.7   
3  NaN    1  37886  69259000BY0093      Toit plan      2.7      207.3   
4  NaN    1  37887  69259000BY0093  Toit non plan      4.4      207.7   

   ztoitmaxi  hmax  millesime  _predicate  \
0      219.8   8.4       2018  INTERSECTS   
1      231.8  20.2       2018  INTERSECTS   
2      209.1   5.8       2018  INTERSECTS   
3      207.3   2.7       2018  INTERSECTS   
4      209.1   5.8       2018  INTERSECTS   

                                            geometry  
0  POLYGON ((1844725.515 5168325.123, 1844724.025...  
1  POLYGON ((1845761.464 5167720.974, 1845759.953...  
2  POLYGON ((1845879.239 5168615.414, 1845879.364...  
3  POLYGON ((1845882.089 5168610

In [40]:
# Define function to transform geopackage rows into guess work file inputs
def to_inputs(row):
    
    geom_geojson = mapping(row["geometry"])
    geom = GEOSGeometry(json.dumps(geom_geojson))
    geom.srid = srid
    geom.transform(4326)
    
    
    
    
    return {
        'ext_id': row["id"],
        'polygon': geom.json
    }

In [41]:
inputs = []

for idx, row in gdf.iterrows():
    inputs.append(to_inputs(row))

print('Sneak peak of row to inputs transformation')
print(inputs[:5])


Sneak peak of row to inputs transformation
[{'ext_id': 42119, 'polygon': '{ "type": "Polygon", "coordinates": [ [ [ 4.858588636054994, 45.699804623187475 ], [ 4.8585692463174, 45.699796842705517 ], [ 4.858548765629505, 45.699803664169458 ], [ 4.858588636054994, 45.699804623187475 ] ] ] }'}, {'ext_id': 37869, 'polygon': '{ "type": "Polygon", "coordinates": [ [ [ 4.871705289737563, 45.69415180077258 ], [ 4.871685345266418, 45.694135788240963 ], [ 4.871671403680955, 45.694144703213041 ], [ 4.87169181843724, 45.694160722401506 ], [ 4.871705289737563, 45.69415180077258 ] ] ] }'}, {'ext_id': 37885, 'polygon': '{ "type": "Polygon", "coordinates": [ [ [ 4.873487184628251, 45.702172719054019 ], [ 4.873484741363241, 45.702052092520738 ], [ 4.873447644632615, 45.702026493215101 ], [ 4.87345061271473, 45.702173082510143 ], [ 4.873487184628251, 45.702172719054019 ] ] ] }'}, {'ext_id': 37886, 'polygon': '{ "type": "Polygon", "coordinates": [ [ [ 4.873522251387816, 45.702126954725138 ], [ 4.873552334